# 🎨 Multi-Format Report Generation (Phase 4)

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4CAF50;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Advanced<br>
<b>Estimated Time:</b> 20 minutes<br>
<b>Prerequisites:</b> Phase 4 implementation complete<br>
<b>Features:</b> PDF, Markdown, HTML, JSON adapters<br>
<b>Dataset:</b> Breast Cancer (sklearn)
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Use Phase 4 adapters (PDF, Markdown, HTML, JSON)
- ✅ Generate reports in multiple formats from same data
- ✅ Understand presentation-agnostic domain model
- ✅ Customize each format appropriately
- ✅ Choose right format for each use case

---

## 📚 Table of Contents

1. [Setup](#setup)
2. [Run Test and Get Results](#test)
3. [Create Domain Model](#domain)
4. [Generate PDF Report](#pdf)
5. [Generate Markdown Report](#markdown)
6. [Generate HTML Report](#html)
7. [Generate JSON Export](#json)
8. [Compare All Formats](#compare)
9. [Conclusion](#conclusion)

<a id="setup"></a>
## 1. 🛠️ Setup

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from datetime import datetime

# sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DeepBridge Core
from deepbridge import DBDataset, Experiment

# Phase 4: New Adapters
from deepbridge.core.experiment.report.adapters import (
    PDFAdapter,
    MarkdownAdapter,
    HTMLAdapter,
    JSONAdapter
)

# Phase 4: Domain Model
from deepbridge.core.experiment.report.domain import (
    Report,
    ReportMetadata,
    ReportType,
    ReportSection,
    Metric,
    MetricType,
    ChartSpec,
    ChartType
)

# Settings
warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Create output directory
output_dir = Path('outputs/multi_format_reports')
output_dir.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")
print(f"📁 Output directory: {output_dir}")
print(f"\n🎉 Phase 4 adapters loaded successfully!")
print(f"   • PDFAdapter")
print(f"   • MarkdownAdapter")
print(f"   • HTMLAdapter")
print(f"   • JSONAdapter")

<a id="test"></a>
## 2. 🔬 Run Test and Get Results

In [ ]:
print("📊 Preparing experiment...\n")

# Load and prepare data
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

# Train model
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# Create experiment
dataset = DBDataset(data=df, target_column='target', model=model, test_size=0.2, random_state=RANDOM_STATE)
exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    test_size=0.2,
    random_state=RANDOM_STATE
)

# Run uncertainty test
print("🔬 Running uncertainty test...")
uncertainty_result = exp.run_test('uncertainty', config='quick')

print("\n✅ Test complete! Ready to generate reports in multiple formats.")

<a id="domain"></a>
## 3. 🏗️ Create Domain Model (Presentation-Agnostic)

The key innovation in Phase 4 is the **presentation-agnostic domain model**.
We define **WHAT** to display, not **HOW** to display it.

In [ ]:
print("🏗️  Creating presentation-agnostic report domain model...\n")

# Step 1: Create metadata
metadata = ReportMetadata(
    model_name="RandomForest Classifier",
    model_type="binary_classification",
    test_type=ReportType.UNCERTAINTY,
    created_at=datetime.now(),
    dataset_name="Breast Cancer Wisconsin",
    dataset_size=len(df),
    tags=["medical", "uncertainty", "demo"]
)

print("✅ Metadata created")

# Step 2: Create report
report = Report(
    metadata=metadata,
    title="Uncertainty Analysis Report",
    subtitle="Breast Cancer Diagnosis Model - Multi-Format Demo",
    introduction="This report demonstrates Phase 4 multi-format generation."
)

print("✅ Report container created")

# Step 3: Add summary metrics
report.add_summary_metric(
    Metric(
        name="Test Accuracy",
        value=model.score(X_test, y_test),
        type=MetricType.PERCENTAGE,
        description="Model accuracy on test set",
        threshold=0.90,
        is_primary=True
    )
)

report.add_summary_metric(
    Metric(
        name="Coverage @90%",
        value=0.92,
        type=MetricType.PERCENTAGE,
        description="Prediction interval coverage",
        is_primary=True
    )
)

print(f"✅ Added {len(report.summary_metrics)} summary metrics")

# Step 4: Add sections
results_section = ReportSection(
    id="test_results",
    title="Uncertainty Test Results",
    description="Detailed analysis of model uncertainty quantification"
)

results_section.add_metric(
    Metric(
        name="Mean Interval Width",
        value=0.234,
        type=MetricType.SCALAR,
        description="Average width of prediction intervals"
    )
)

results_section.add_metric(
    Metric(
        name="Calibration Error",
        value=0.0234,
        type=MetricType.SCALAR,
        description="Expected Calibration Error (ECE)"
    )
)

# Add a chart spec (not rendered yet)
results_section.add_chart(
    ChartSpec(
        id="coverage_chart",
        type=ChartType.COVERAGE,
        title="Coverage vs Confidence Level",
        description="Analysis of prediction interval coverage",
        data={
            "alphas": [0.1, 0.2, 0.3],
            "coverage": [0.91, 0.82, 0.73],
            "expected": [0.90, 0.80, 0.70]
        }
    )
)

report.add_section(results_section)

print(f"✅ Added section with {len(results_section.metrics)} metrics and {len(results_section.charts)} chart")

print("\n🎉 Domain model complete!")
print(f"   Report: {report.title}")
print(f"   Sections: {len(report.sections)}")
print(f"   Summary Metrics: {len(report.summary_metrics)}")
print(f"   Total Metrics: {len(report.get_all_metrics())}")
print(f"\n💡 This model is format-independent - we can render it to any format!")

<a id="pdf"></a>
## 4. 📕 Generate PDF Report

In [ ]:
print("📕 Generating PDF report...\n")

# Create PDF adapter
pdf_adapter = PDFAdapter(
    theme="professional",
    page_size="A4"
)

# Render to PDF
pdf_bytes = pdf_adapter.render(report)

# Save to file
pdf_path = output_dir / 'uncertainty_report.pdf'
pdf_adapter.save_to_file(pdf_bytes, str(pdf_path))

print(f"✅ PDF report generated!")
print(f"   Path: {pdf_path}")
print(f"   Size: {len(pdf_bytes) / 1024:.1f} KB")
print(f"\n📋 PDF features:")
print(f"   • Print-optimized CSS")
print(f"   • Page breaks")
print(f"   • Static images for charts")
print(f"   • Professional layout")
print(f"   • A4 page size")

<a id="markdown"></a>
## 5. 📝 Generate Markdown Report

In [ ]:
print("📝 Generating Markdown report...\n")

# Create Markdown adapter
md_adapter = MarkdownAdapter(
    include_toc=True,
    heading_level_start=1,
    chart_placeholder="chart"  # or "link" or "ignore"
)

# Render to Markdown
markdown = md_adapter.render(report)

# Save to file
md_path = output_dir / 'uncertainty_report.md'
md_adapter.save_to_file(markdown, str(md_path))

print(f"✅ Markdown report generated!")
print(f"   Path: {md_path}")
print(f"   Size: {len(markdown) / 1024:.1f} KB")
print(f"\n📋 Markdown features:")
print(f"   • Table of Contents")
print(f"   • GitHub/GitLab compatible")
print(f"   • Metric tables")
print(f"   • Chart placeholders")
print(f"   • Hierarchical sections")

# Preview first 500 characters
print(f"\n📄 Preview:")
print("=" * 80)
print(markdown[:500] + "...")
print("=" * 80)

<a id="html"></a>
## 6. 🌐 Generate HTML Report

In [ ]:
print("🌐 Generating HTML report...\n")

# Note: HTMLAdapter requires template_manager and asset_manager
# For now, we'll show the concept

print("💡 HTML Generation Concept:")
print("""
# Create HTML adapter (requires template/asset managers)
html_adapter = HTMLAdapter(
    template_manager=template_mgr,
    asset_manager=asset_mgr,
    theme="default"
)

# Render to HTML
html = html_adapter.render(report)

# Save
with open('report.html', 'w') as f:
    f.write(html)
""")

print("\n📋 HTML features:")
print("   • Interactive charts (Plotly)")
print("   • Responsive design")
print("   • Custom themes")
print("   • Embedded assets")
print("   • JavaScript interactivity")

<a id="json"></a>
## 7. 📊 Generate JSON Export

In [ ]:
print("📊 Generating JSON export...\n")

# Create JSON adapter
json_adapter = JSONAdapter(indent=2)

# Render to JSON
json_str = json_adapter.render(report)

# Save to file
json_path = output_dir / 'uncertainty_report.json'
with open(json_path, 'w') as f:
    f.write(json_str)

print(f"✅ JSON export generated!")
print(f"   Path: {json_path}")
print(f"   Size: {len(json_str) / 1024:.1f} KB")
print(f"\n📋 JSON features:")
print(f"   • Machine-readable")
print(f"   • API-friendly")
print(f"   • Programmatic access")
print(f"   • Type information preserved")

# Preview structure
import json
json_data = json.loads(json_str)

print(f"\n📄 JSON Structure:")
print(f"   Keys: {list(json_data.keys())}")
print(f"   Metadata: {list(json_data['metadata'].keys())[:5]}...")
print(f"   Sections: {len(json_data['sections'])}")
print(f"   Summary Metrics: {len(json_data['summary_metrics'])}")

<a id="compare"></a>
## 8. 📊 Compare All Formats

In [ ]:
print("📊 Format Comparison\n")
print("=" * 90)

# Check file sizes
files = {
    'PDF': pdf_path,
    'Markdown': md_path,
    'JSON': json_path
}

comparison_data = []
for format_name, path in files.items():
    if path.exists():
        size_kb = path.stat().st_size / 1024
        comparison_data.append({
            'Format': format_name,
            'File Size': f"{size_kb:.1f} KB",
            'Generated': '✅',
            'Path': path.name
        })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

print("\n📋 Use Case Recommendations:")
print("\n1. PDF - Best for:")
print("   • Regulatory submissions")
print("   • Archival records")
print("   • Stakeholder presentations")
print("   • Printing")

print("\n2. Markdown - Best for:")
print("   • GitHub/GitLab documentation")
print("   • Static site generators")
print("   • Version control")
print("   • Jupyter notebooks")

print("\n3. HTML - Best for:")
print("   • Interactive exploration")
print("   • Web dashboards")
print("   • Development/testing")
print("   • Rich visualizations")

print("\n4. JSON - Best for:")
print("   • API responses")
print("   • Programmatic access")
print("   • Data pipelines")
print("   • Custom processing")

### Advanced: Generate All Formats at Once

In [ ]:
print("🚀 Advanced: Generate all formats in one go!\n")

def generate_all_formats(report, output_dir, base_name="report"):
    """Generate report in all available formats."""
    output_dir = Path(output_dir)
    results = {}
    
    # PDF
    pdf_adapter = PDFAdapter()
    pdf_bytes = pdf_adapter.render(report)
    pdf_path = pdf_adapter.save_to_file(pdf_bytes, str(output_dir / f"{base_name}.pdf"))
    results['PDF'] = pdf_path
    
    # Markdown
    md_adapter = MarkdownAdapter(include_toc=True)
    markdown = md_adapter.render(report)
    md_path = md_adapter.save_to_file(markdown, str(output_dir / f"{base_name}.md"))
    results['Markdown'] = md_path
    
    # JSON
    json_adapter = JSONAdapter(indent=2)
    json_str = json_adapter.render(report)
    json_path = output_dir / f"{base_name}.json"
    with open(json_path, 'w') as f:
        f.write(json_str)
    results['JSON'] = str(json_path)
    
    return results

# Use it
all_outputs = generate_all_formats(report, output_dir, "complete_report")

print("✅ All formats generated:")
for format_name, path in all_outputs.items():
    print(f"   {format_name}: {path}")

print("\n💡 This function can be used in automation pipelines!")

<a id="conclusion"></a>
## 9. 🎓 Conclusion

### What You Learned

- ✅ **Phase 4 Adapters** - PDF, Markdown, HTML, JSON
- ✅ **Domain Model** - Presentation-agnostic report structure
- ✅ **Multi-Format Generation** - Same data, multiple formats
- ✅ **Format Comparison** - Strengths of each format
- ✅ **Automation** - Generate all formats at once

### Key Innovations in Phase 4

1. **Separation of Concerns**
   - Domain model (WHAT) vs Adapter (HOW)
   - Easy to add new formats

2. **Type Safety**
   - Pydantic validation
   - Compile-time checks
   - Auto-completion

3. **Consistency**
   - Same API for all formats
   - Predictable behavior
   - Easy to test

### Best Practices

1. **Choose format based on audience**
   - Technical: Markdown, JSON
   - Business: PDF, HTML
   - Regulatory: PDF

2. **Generate multiple formats**
   - PDF for records
   - Markdown for docs
   - JSON for automation

3. **Automate in CI/CD**
   - Generate on every test run
   - Archive PDF versions
   - Update Markdown docs

---

**🎉 Congratulations! You've mastered Phase 4 multi-format report generation!**

Next: Try async batch generation (notebook 03)!